In [1]:
from portfolio_analyzer.stocks_data_loader import yahoo2pandas
from portfolio_analyzer.optimizers import approximated_max_kelly, minimal_variance, risk_parity
from portfolio_analyzer.ameritrade_rebalance_utils import AmeritradeRebalanceUtils

In [6]:
tda = AmeritradeRebalanceUtils()
# create td_client_auth.json with {"client_id": "ameritrade_supplied_client_id", "callback_url": "ameritrade_callback_url_you_set_up"}
tda.auth('../td_tokens.json', '../td_client_auth.json')

tickers = ['VTI', 'TLT', 'IAU']
data = yahoo2pandas(tickers, frequency='weekly')

Please go to URL provided authorize your account: I'M OBFUSCATING THIS FOR SECURITY REASONS

In [7]:
risk_parity(data)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.0130100430612985e-15
            Iterations: 20
            Function evaluations: 80
            Gradient evaluations: 20


,VTI,TLT,IAU
0,0.324121,0.426535,0.249344


In [8]:
minimal_variance(data)

,VTI,TLT,IAU
0,0.336505,0.520083,0.143411


In [9]:
approximated_max_kelly(data)

,VTI,TLT,IAU
0,0.339573,0.506163,0.154265


In [10]:
from portfolio_analyzer.balancer import Rebalance
from portfolio_analyzer.utils import portfolio2dic

In [25]:
portfolio_exposure = tda.get_portfolio()
portfolio_exposure

extra_cash = 123. # Cash you want add or subtract from your portfolio. 

optimal_weights = portfolio2dic(approximated_max_kelly(data))

In [26]:
rebalancer = Rebalance(portfolio_exposure, optimal_weights, extra_cash)

In [27]:
new_exposure, current_balance = rebalancer.compute()

In [28]:
new_exposure, current_balance

({'VTI': 10, 'TLT': 22, 'IAU': 56}, 5921.210090637207)

In [29]:
diff = rebalancer.diff(new_exposure)
diff

{'IAU': 23, 'TLT': 0, 'VTI': -1}

In [30]:
dry_run = tda.place_orders_dry_run(diff)
dry_run

{'IAU': {'instruction': 'BUY', 'qty': 23, 'money_movement': -372.14},
 'TLT': {'instruction': 'SELL', 'qty': 0, 'money_movement': -0.0},
 'VTI': {'instruction': 'SELL', 'qty': 1, 'money_movement': 200.75}}

In [31]:
# tda.place_orders(dry_run)